## setting

In [67]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("/content/drive/My Drive/LCNN/")

import os
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt



import imp
import importlib
import LCNN
imp.reload(LCNN)
from LCNN import adjust,accuracy,one_hot
device = torch.device('cuda')


Mounted at /content/drive


In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
batchsize=64

trainset = datasets.FashionMNIST('/content/drive/My Drive/data', download=True, train=True, transform=transform)
valset = datasets.FashionMNIST('/content/drive/My Drive/data', download=True, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=False)
valloader = torch.utils.data.DataLoader(valset, batch_size=batchsize, shuffle=False)

In [91]:
# Device configuration
device = torch.device('cuda')
# Hyper-parameters 
input_size = 784

hidden_size1 = 1000

num_classes = 10
batch_size = 64

import torch.nn.functional as F
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


In [ ]:

for p in [0,10]:
    learning_rate = 0.01
    torch.manual_seed(0)
    device = torch.device('cuda')
    model = NeuralNet().to(device)
    ad=False
    if p>20000000:
        ad=True
    if ad:
        dataiter = iter(train_loader)
        images, labels = dataiter.next()
        with torch.no_grad():
            output = model(images.to(device)).to(device)
            adjust(model,torch.flatten(activation['dropout1'], 1),threshold_u=10,threshold_l=0.01,scale=0.8,ln=1,oflag=2)
            
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    #criterion = nn.MSELoss()
    num_epochs=100
    dev=100
    losslist=[]


    totalloss=[]
    aclist=[]

    count=0
    dd=p
    de=1
    lo=0
    aco=0
    ts=len(train_loader)
    for epoch in range(num_epochs):
        loss_epoch=0
        for i, (images, labels) in enumerate(train_loader):  
            if i>ts:
                break

            images = images.to(device)
            outputs = model(images).to(device)
            

            #labels =one_hot(labels,num_classes).to(device)
            loss = criterion(outputs, labels).to(device)
            if count==p and p>0:
                with torch.no_grad():
                    x = images.to(device)
                    x = model.conv1(x)
                    x = F.relu(x)
                    x = model.conv2(x)
                    x = F.relu(x)
                    x = F.max_pool2d(x, 2)
                    x = torch.flatten(x, 1)
                    x=x+torch.tensor(0.00001)
                    adjust(model,x,threshold_u=20,threshold_l=0.01,scale=1.0,ln=1,oflag=2)
                    
            totalloss.append(loss.item())
            if count<10 or ((count>p-10) and (count<p+10)) :
                print(count,loss.item())
            loss_epoch=loss_epoch+loss.item()       
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            count=count+1
                        
            
        #loss_epoch=loss_epoch/len(train_loader)
        loss_epoch=loss_epoch/ts
        losslist.append(loss_epoch)
        #ac=(accuracy(model,valloader))
        aclist.append(ac)
        
        print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss_epoch))
        if loss_epoch<0.001 or np.abs(loss_epoch-lo)<0.00001: #or aco>ac:
            break

        lo=loss_epoch
        aco=ac
    
    out.append([losslist,totalloss,aclist])

0 2.3158230781555176
1 2.2912697792053223
2 2.2925775051116943
3 2.2922356128692627
4 2.2570509910583496
5 2.248633623123169
6 2.225332498550415
7 2.2200686931610107
8 2.178046703338623
9 2.1480090618133545
Epoch [1/100], Loss: 0.4765
Epoch [2/100], Loss: 0.2913
Epoch [3/100], Loss: 0.2388
Epoch [4/100], Loss: 0.2029
Epoch [5/100], Loss: 0.1724
Epoch [6/100], Loss: 0.1450
Epoch [7/100], Loss: 0.1212
Epoch [8/100], Loss: 0.0993
Epoch [9/100], Loss: 0.0820
Epoch [10/100], Loss: 0.0726
Epoch [11/100], Loss: 0.0647
Epoch [12/100], Loss: 0.0513
Epoch [13/100], Loss: 0.0417
Epoch [14/100], Loss: 0.0360
Epoch [15/100], Loss: 0.0296
Epoch [16/100], Loss: 0.0254
Epoch [17/100], Loss: 0.0229
Epoch [18/100], Loss: 0.0207
Epoch [19/100], Loss: 0.0189
Epoch [20/100], Loss: 0.0169
Epoch [21/100], Loss: 0.0123
Epoch [22/100], Loss: 0.0108
Epoch [23/100], Loss: 0.0085
Epoch [24/100], Loss: 0.0073
Epoch [25/100], Loss: 0.0071
Epoch [26/100], Loss: 0.0058
Epoch [27/100], Loss: 0.0060
Epoch [28/100], Los

In [88]:
0 2.3158230781555176
1 2.2912697792053223
2 2.2925775051116943
3 2.2922356128692627
4 2.2570509910583496
5 2.248633623123169
6 2.225332498550415
7 2.2200686931610107
8 2.178046703338623
9 2.1480090618133545
241 0.5762401223182678
242 0.4286177456378937
243 0.4594425559043884
244 0.45815029740333557
245 0.4146536886692047
246 0.4354739785194397
247 0.21415457129478455
248 0.47104939818382263
249 0.46312057971954346
tensor([ 8.0556,  8.5518, 10.0000,  0.1622,  2.4447, 10.0000, 10.0000,  0.9244,
        10.0000,  0.2124,  1.4809,  2.8393,  0.2816,  0.0900,  0.2754,  1.3408,
         0.8896, 10.0000,  3.3678,  2.0077,  1.3368,  0.1678,  0.3402,  1.9790,
         6.6829,  1.5271,  8.4547,  2.9978, 10.0000,  0.8262,  0.0303,  8.8586,
        10.0000,  3.8690, 10.0000, 10.0000,  2.2984, 10.0000,  0.3746,  3.6729,
         0.6457,  1.2205,  0.0462, 10.0000,  5.0670,  0.1810, 10.0000,  4.0644,
         0.2693,  5.2688,  1.9039,  0.7479, 10.0000, 10.0000,  0.3088,  0.4793,
        10.0000,  0.0110, 10.0000, 10.0000,  0.1260,  0.0113,  9.7281,  6.3551,
         1.0262,  3.6866,  0.6881,  1.9280,  5.4587, 10.0000,  1.4879,  1.3132,
        10.0000, 10.0000,  0.8267,  2.7750,  5.7247,  1.7339,  0.5366,  0.4354,
         3.8174,  1.8467, 10.0000,  6.9977,  0.9325,  0.2555,  0.4989,  1.5725,
        10.0000,  1.4343,  1.6061,  0.8329, 10.0000,  0.1240,  9.0032,  5.2969,
         1.9218,  8.5522, 10.0000,  0.9724,  2.1029,  1.7609, 10.0000,  1.9047,
        10.0000,  0.1465,  2.1651,  2.6199,  0.6717,  0.2228,  0.4108,  1.4500,
        10.0000,  0.0277, 10.0000,  0.4592,  0.8250,  0.3589,  0.6367,  0.2013,
        10.0000,  8.4030, 10.0000,  3.4183,  1.9974,  0.0395,  8.4609,  0.2893])
Adjusting Layer 1, Kernel Nodes: 44, Adptive Nodes84
250 0.5872054696083069
251 0.8813714981079102
252 0.672589898109436
253 0.9812057614326477
254 0.6765425801277161
255 0.6622276306152344
256 0.7763565182685852
257 0.47490984201431274
258 0.9602660536766052
259 0.5998060703277588
Epoch [1/100], Loss: 0.4786
Epoch [2/100], Loss: 0.2742
Epoch [3/100], Loss: 0.2218
Epoch [4/100], Loss: 0.1861
Epoch [5/100], Loss: 0.1565
Epoch [6/100], Loss: 0.1301
Epoch [7/100], Loss: 0.1081
Epoch [8/100], Loss: 0.0895
Epoch [9/100], Loss: 0.0740
Epoch [10/100], Loss: 0.0660
Epoch [11/100], Loss: 0.0616
Epoch [12/100], Loss: 0.0535
Epoch [13/100], Loss: 0.0452
Epoch [14/100], Loss: 0.0357
Epoch [15/100], Loss: 0.0255
Epoch [16/100], Loss: 0.0213
Epoch [17/100], Loss: 0.0172
Epoch [18/100], Loss: 0.0175
Epoch [19/100], Loss: 0.0157
Epoch [20/100], Loss: 0.0127
Epoch [21/100], Loss: 0.0126
Epoch [22/100], Loss: 0.0110
Epoch [23/100], Loss: 0.0080
Epoch [24/100], Loss: 0.0058

2304

In [81]:
torch.flatten(activation['dropout1'], 1)

KeyError: ignored

In [ ]:
activation['dropout1']

In [40]:
model = NeuralNet().to(device)
model.dropout1.register_forward_hook(get_activation('dropout1'))

In [25]:
model.state_dict()['fc'+str(1)+'.weight']

tensor([[ 0.0027,  0.0100, -0.0051,  ..., -0.0101, -0.0084,  0.0011],
        [ 0.0087,  0.0080,  0.0054,  ..., -0.0030,  0.0035, -0.0092],
        [-0.0025,  0.0088, -0.0026,  ..., -0.0056, -0.0057, -0.0089],
        ...,
        [ 0.0054,  0.0016,  0.0035,  ...,  0.0038, -0.0059, -0.0008],
        [-0.0080, -0.0052, -0.0022,  ...,  0.0052,  0.0078, -0.0054],
        [-0.0047, -0.0047,  0.0020,  ...,  0.0097, -0.0075,  0.0002]])

In [26]:
images

tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],


        [[[-1.0000, -0.9843, -1.0000,  ..., -0.9922, -1.0000, -1.0000],
          [-1.0000, -1.000

In [30]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
ln=1

In [31]:
hs=len(model.state_dict()['fc'+str(ln)+'.weight'])

In [32]:
hs

128

In [52]:
len(activation['dropout1'][0])

64

In [41]:
output = model(images.to(device)).to(device)

In [43]:
adjust(model,torch.flatten(activation['dropout1'], 1),threshold_u=10,threshold_l=0.01,scale=0.8,ln=1,oflag=2)

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan])
Adjusting Layer 1, Kernel Nodes: 0, Adptive Nodes0


In [54]:
 L=len(images)

In [55]:
L

64

In [47]:
hs=len(model.state_dict()['fc'+str(ln)+'.weight'])

In [48]:
hs

128

In [63]:
d_=torch.zeros(hs).to(device)

In [64]:
for i in range(hs):
            d=0
            for k in range(L):
                image=images[k]
                l=-model.state_dict()['fc'+str(ln)+'.weight'][i]/model.state_dict()['fc'+str(ln)+'.bias'][i]
                image=torch.min(image,torch.tensor(0.0001))
                x_k=1/image.to(device)

                d_k=(torch.abs(torch.sum(l*x_k)-1.0)/torch.norm(x_k))
                d=d+d_k


In [53]:
images=torch.flatten(activation['dropout1'], 1)

In [65]:
print(d_)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])


In [66]:
print(image)

tensor([1.0000e-04, 1.0000e-04, 1.0000e-04,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00])


In [71]:
torch.flatten(activation['dropout1'], 1)

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])